In [ ]:
import sys
sys.path.insert(0, "../")

import numpy as np
import pandas as pd
from ctgan.synthesizers.ctgan import CTGANSynthesizer
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle

from src.metrics import *
from src.metrics import eval_plugin
from src.synthesizer import fit_ctgan
from src.utils import *

seed = 0
seed_everything(seed)

dataset_name = "drug"


# Load data

In [ ]:
from src.data_loader import load_drug_dataset

X, y, Data = load_drug_dataset()


In [ ]:
X_train, X_oracle = train_test_split(Data, test_size=0.5, random_state=seed)

X_train, X_test = train_test_split(X_train, test_size=0.2, random_state=seed)

X_train, X_hp = train_test_split(X_train, test_size=0.1, random_state=seed)


# Train Generators

In [ ]:
import pickle

ctgans = []
trials = []
params_list = []

discrete_columns = [
    "Age",
    "Gender",
    "Education",
    "Country",
    "Ethnicity",
    "Impulsive",
    "SS",
    "Alcohol",
    "Amphet",
    "Amyl",
    "Benzos",
    "Cannabis",
    "Coke",
    "Crack",
    "Ecstasy",
    "Heroin",
    "Ketamine",
    "Legalh",
    "LSD",
    "Meth",
    "Mushrooms",
    "VSA",
    "y",
]

use_trained_model = False

pd.options.mode.chained_assignment = None
hp_sweep = False
if use_trained_model == False:
    if hp_sweep:
        import optuna
        from optuna.samplers import TPESampler

        # Optimize HPS
        def gan_objective(trial):

            discrete_columns = [
                "Age",
                "Gender",
                "Education",
                "Country",
                "Ethnicity",
                "Impulsive",
                "SS",
                "Alcohol",
                "Amphet",
                "Amyl",
                "Benzos",
                "Cannabis",
                "Coke",
                "Crack",
                "Ecstasy",
                "Heroin",
                "Ketamine",
                "Legalh",
                "LSD",
                "Meth",
                "Mushrooms",
                "VSA",
                "y",
            ]

            learning_rate = trial.suggest_categorical(
                "learning_rate", [2e-4, 2e-5, 2e-6]
            )
            embedding_dim = trial.suggest_categorical("embedding_dim", [64, 128, 256])
            epochs = trial.suggest_categorical("epochs", [200, 300, 500])

            ctgan = fit_ctgan(
                data=X_test,
                epochs=epochs,
                learning_rate=learning_rate,
                embedding_dim=embedding_dim,
                discrete_columns=discrete_columns,
            )
            D_fake, _ = ctgan.sample(X_hp.shape[0], shift=False)
            metric = MaximumMeanDiscrepancy
            trial_results = eval_plugin(
                metric,
                GenericDataLoader(X_hp.astype(float)),
                GenericDataLoader(D_fake.astype(float)),
            )
            trials.append(trial_results)
            print(
                eval_plugin(
                    WassersteinDistance,
                    GenericDataLoader(X_hp.astype(float)),
                    GenericDataLoader(D_fake.astype(float)),
                )
            )

            print(f"HPS = Params: {trial.params} | Score: {trial_results[0]}")
            return trial_results[0]["joint"]

        gan_study = optuna.create_study(direction="minimize", sampler=TPESampler())
        gan_study.optimize(gan_objective, show_progress_bar=True, n_trials=10)
        print("Best parameters:", gan_study.best_params)

        ctgan = fit_ctgan(
            data=X_test,
            epochs=gan_study.best_params["epochs"],
            learning_rate=gan_study.best_params["learning_rate"],
            embedding_dim=gan_study.best_params["embedding_dim"],
            discrete_columns=discrete_columns,
        )
        ctgans.append(deepcopy(ctgan))

    else:

        from copy import deepcopy
        from tqdm import tqdm

        for i in tqdm(range(10)):

            best_params = {"learning_rate": 0.0002, "embedding_dim": 256, "epochs": 100}
            ctgan = fit_ctgan(
                data=X_train,
                epochs=best_params["epochs"],
                learning_rate=best_params["learning_rate"],
                embedding_dim=best_params["embedding_dim"],
                seed=seed + i,
                discrete_columns=discrete_columns,
            )

            D_fake, _ = ctgan.sample(X_hp.shape[0], shift=False)

            trial_results = eval_plugin(
                MaximumMeanDiscrepancy,
                GenericDataLoader(X_hp.astype(float)),
                GenericDataLoader(D_fake.astype(float)),
            )
            trials.append(trial_results)
            params_list.append(best_params)
            ctgans.append(deepcopy(ctgan))

        # save each ctgan model in ctgan_list
        for idx, ctgan_save in enumerate(ctgans):
            ctgan_save.save(f"../models/ctgan_{dataset_name}_{idx+1}")

        trials_list = [trials[idx][0]["joint"] for idx in range(len(trials))]
        ctgan_idx = trials_list.index(min(trials_list))

        ctgan = ctgans[ctgan_idx]

    ctgan.save(f"../models/ctgan_{dataset_name}")

    # pickle best params
    with open(f"../models/ctgan_{dataset_name}_params.pkl", "wb") as f:
        pickle.dump(params_list[ctgan_idx], f)


else:
    # Load best_params
    with open(f"../models/ctgan_{dataset_name}_params.pkl", "rb") as f:
        best_params = pickle.load(f)

    ctgan = CTGANSynthesizer(
        embedding_dim=best_params["embedding_dim"],
        generator_dim=(256, 256),
        discriminator_dim=(256, 256),
        generator_lr=best_params["learning_rate"],
        generator_decay=1e-6,
        discriminator_lr=best_params["learning_rate"],
        discriminator_decay=1e-6,
        batch_size=500,
        discriminator_steps=1,
        log_frequency=True,
        verbose=False,
        epochs=best_params["epochs"],
        pac=10,
        cuda=True,
    )

    ctgan = ctgan.load(f"../models/ctgan_{dataset_name}")


# Helpers

In [ ]:
from copy import deepcopy

import numpy as np
from fairlearn.metrics import demographic_parity_ratio as dp_ratio
from fairlearn.metrics import equalized_odds_ratio as eo_ratio
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm


def evaluate_models(models, data, column_metric, sensitive="sex"):
    performance = {"acc": {}, "f1": {}, "eo": {}, "dp": {}}

    for model_name, model in models.items():
        if data.shape[0] == 0:
            performance["acc"][model_name] = 0
            performance["f1"][model_name] = 0
            performance["eo"][model_name] = 0
            performance["dp"][model_name] = 0
            continue

        y_pred = model.predict(data.drop("y", axis=1))
        accuracy = accuracy_score(data["y"], y_pred)
        F1 = f1_score(data["y"], y_pred)

        try:
            eo_score = eo_ratio(
                data["y"], y_pred, sensitive_features=data[sensitive].values
            )
        except:
            eo_score = 0

        try:
            dp_score = dp_ratio(
                data["y"], y_pred, sensitive_features=data[sensitive].values
            )
        except:
            dp_score = 0

        performance["acc"][model_name] = accuracy
        performance["f1"][model_name] = F1
        performance["eo"][model_name] = eo_score
        performance["dp"][model_name] = dp_score

    return performance


def run_analysis(
    column_metric,
    ctgan,
    Data,
    X_train,
    X_test,
    X_oracle,
    trained_model_dict,
    random_state=0,
    n_samples=1000,
):
    # Initialization
    groups = list(np.unique(Data[column_metric]))
    X_train, X_oracle = train_test_split(Data, test_size=0.65, random_state=0)
    X_train, X_test = train_test_split(
        X_train, test_size=0.2, random_state=random_state
    )
    test_dataset = deepcopy(X_test)

    (
        test_results,
        test_samples,
        oracle_results,
        oracle_samples,
        synth_results,
        synth_samples,
        aug_results,
        aug_samples,
    ) = ({}, {}, {}, {}, {}, {}, {}, {})

    for group in tqdm(groups):
        total_samples = (
            X_test[X_test[column_metric] == 0].shape[0] if group == 0 else n_samples
        )
        test_data = X_test[X_test[column_metric] == group]
        oracle_data = X_oracle[X_oracle[column_metric] == group]

        # Dtest
        test_results[group], test_samples[group] = (
            evaluate_models(trained_model_dict, test_data, column_metric),
            test_data.shape[0],
        )
        oracle_results[group], oracle_samples[group] = (
            evaluate_models(trained_model_dict, oracle_data, column_metric),
            oracle_data.shape[0],
        )

        # Synthetic data sampling
        synth_data, _ = ctgan.sample(
            1, shift=False, condition_column=column_metric, condition_value=group
        )
        count = 0
        while synth_data.shape[0] <= total_samples:
            tmp_df = ctgan.sample(
                n_samples,
                shift=False,
                condition_column=column_metric,
                condition_value=group,
            )[0]
            synth_data = synth_data.append(tmp_df[tmp_df[column_metric] == group])
            count += 1
        synth_results[group], synth_samples[group] = (
            evaluate_models(trained_model_dict, synth_data, column_metric),
            synth_data.shape[0],
        )

        # Augmented data evaluation
        aug_data = pd.concat([test_data, synth_data])
        aug_results[group], aug_samples[group] = (
            evaluate_models(trained_model_dict, aug_data, column_metric),
            aug_data.shape[0],
        )

    return (
        test_results,
        test_samples,
        oracle_results,
        oracle_samples,
        synth_results,
        synth_samples,
        aug_results,
        aug_samples,
        test_dataset,
    )


# Train the downstream predictors

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

column_metric = "Ethnicity"

model_dict = {
    "mlp": MLPClassifier(random_state=seed),
    "knn": KNeighborsClassifier(),
    "dt": DecisionTreeClassifier(random_state=seed),
    "rf": RandomForestClassifier(random_state=seed),
    "gbc": GradientBoostingClassifier(random_state=seed),
    "bag": BaggingClassifier(random_state=seed),
    "ada": AdaBoostClassifier(random_state=seed),
    "svm": SVC(random_state=seed),
    "lr": LogisticRegression(random_state=seed),
}

print("training baseline models")
trained_model_dict = train_models(X_train, model_dict)


In [ ]:
trials_list = [trials[idx][0]["joint"] for idx in range(len(trials))]
ctgan_idx = trials_list.index(min(trials_list))
ctgan_idx=0
ctgan = ctgans[ctgan_idx]
ctgan_idx


# Run analysis

In [ ]:
from tqdm import tqdm

oracle_acc_list = []
oracle_samples_list = []
test_acc_list = []
test_samples_list = []
synth_acc_list = []
synth_samples_list = []
aug_acc_list = []
aug_samples_list = []
final_test_dataset = []


n_runs = 5
ns = 2000

for i in tqdm(range(n_runs)):

    done = False
    tries = 0

    while done == False:
        try:
            (
                test_accs,
                test_samples,
                oracle_accs,
                oracle_samples,
                synth_accs,
                synth_samples,
                aug_accs,
                aug_samples,
                test_dataset,
            ) = run_analysis(
                column_metric=column_metric,
                Data=Data,
                X_train=X_train,
                X_test=X_test,
                X_oracle=X_oracle,
                ctgan=ctgan,
                trained_model_dict=trained_model_dict,
                random_state=i * 100,
                n_samples=ns,
            )
            done = True
            final_test_dataset.append(test_dataset)
        except Exception:
            import traceback

            print(traceback.format_exc())
            tries += 1

            if tries > 1:
                done = True
            continue

    test_acc_list.append(test_accs)
    test_samples_list.append(test_samples)
    oracle_acc_list.append(oracle_accs)
    oracle_samples_list.append(oracle_samples)
    synth_acc_list.append(synth_accs)
    synth_samples_list.append(synth_samples)
    aug_acc_list.append(aug_accs)
    aug_samples_list.append(aug_samples)


# Process results & save

In [ ]:
from brmp import brm
from brmp.numpyro_backend import backend as numpyro
from brmp.pyro_backend import backend as pyro_backend

backend = pyro_backend

metrics = ["acc"]
models = list(trained_model_dict.keys())

X_test_new = deepcopy(final_test_dataset[0])


for metric in metrics:

    for model in ["rf"]:

        try:
            mbm = []
            for i in range(5):
                seed = i * 100
                X_train, X_oracle = train_test_split(
                    Data, test_size=0.65, random_state=0
                )
                X_train, X_test = train_test_split(
                    X_train, test_size=0.2, random_state=seed
                )
                X_test["S"] = np.max(
                    trained_model_dict[model].predict_proba(X_test.drop("y", axis=1)),
                    axis=1,
                )
                model3 = brm("S ~ Gender + Ethnicity + Education + Country + y", X_test)
                fit3 = model3.fit(backend=backend, seed=seed, iter=1000, warmup=100)
                scores3 = fit3.fitted(what="sample", data=None, seed=seed)
                mbm.append(scores3)

            mbm = np.array(mbm)

            props = [
                count / np.sum(np.unique(Data[column_metric], return_counts=True)[1])
                for count in np.unique(Data[column_metric], return_counts=True)[1]
            ]

            groups = np.sort(list(X_oracle[column_metric].unique()))

            for model_name in [model]:
                print(model_name)
                data_list = []
                idx = 0

                df = pd.DataFrame(columns=["Group", "3S", "3S+", "MBM", "Dtest"])
                df_std = pd.DataFrame(columns=["Group", "3S", "3S+", "MBM", "Dtest"])

                for group in np.argsort(props)[::-1]:
                    if group == 2:
                        group = 4
                    idx += 1
                    mylist = oracle_acc_list
                    oracle_res = np.array(
                        [
                            mylist[i][group][metric][model_name]
                            for i in range(len(mylist))
                        ]
                    )

                    mylist = synth_acc_list
                    synth_res = np.array(
                        [
                            mylist[i][group][metric][model_name]
                            for i in range(len(mylist))
                        ]
                    )

                    mylist = test_acc_list
                    test_res = np.array(
                        [
                            mylist[i][group][metric][model_name]
                            for i in range(len(mylist))
                        ]
                    )

                    mylist = aug_acc_list
                    aug_res = np.array(
                        [
                            mylist[i][group][metric][model_name]
                            for i in range(len(mylist))
                        ]
                    )
                    try:
                        # mbm
                        blm_res = []
                        for i in range(5):
                            y_pred = (np.mean(mbm[i], axis=0) > 0.75).astype(int) # only confident predictions
                            group_ids = np.argwhere(
                                np.array(X_test_new[column_metric] == group).astype(int)
                                == 1
                            )
                            y_pred_group = y_pred[group_ids]
                            y_true_group = X_test_new[
                                X_test_new[column_metric] == group
                            ]["y"]
                            blm_res.append(accuracy_score(y_true_group, y_pred_group))

                        blm_res = np.array(blm_res)

                        mydict = {
                        "Group": f"{idx} ({int(round(props[group]*100,0))}%)",
                        "3S": round(np.mean(np.abs(oracle_res - synth_res)) * 100, 2),
                        "3S+": round(np.mean(np.abs(oracle_res - aug_res)) * 100, 2),
                        "MBM": round(np.mean(np.abs(oracle_res - blm_res)) * 100, 2),
                        "Dtest": round(np.mean(np.abs(oracle_res - test_res)) * 100, 2),
                        }
                        df = df.append(mydict, ignore_index=True)

                        mydict = {
                            "Group": f"{idx} ({int(round(props[group]*100,0))}%)",
                            "3S": round(np.std(np.abs(oracle_res - synth_res)) * 100, 2),
                            "3S+": round(np.std(np.abs(oracle_res - aug_res)) * 100, 2),
                            "MBM": round(np.std(np.abs(oracle_res - blm_res)) * 100, 2),
                            "Dtest": round(np.std(np.abs(oracle_res - test_res)) * 100, 2),
                        }


                        df_std = df_std.append(mydict, ignore_index=True)

                    except Exception:
                        import traceback

                        print(traceback.format_exc())
                        continue

            df.to_csv(f"../results/{dataset_name}_{model}_{metric}.csv")
            df_std.to_csv(f'../results/{dataset_name}_std_{model}_{metric}.csv')

        except Exception as e:
            import traceback

            print(traceback.format_exc())
            print(e)


# Coverage helper

In [ ]:
from copy import deepcopy

import pandas as pd
from brmp import brm
from brmp.numpyro_backend import backend as numpyro
from brmp.pyro_backend import backend as pyro_backend


def get_group(X_test, column_metric, groups, test_group="small"):
    smallest_group = 0
    gsizes = []
    for idx, group in enumerate(groups):
        gsizes.append(X_test[X_test[column_metric] == group].shape[0])

    print(gsizes)
    ns = 1
    for idx, group in enumerate(groups):

        if test_group == "small":
            if idx == 0:
                size_smallest_group = X_test[X_test[column_metric] == 0].shape[0]
                smallest_group = group
                if size_smallest_group < ns:
                    size_smallest_group = 99999
            else:
                if (
                    X_test[X_test[column_metric] == group].shape[0]
                    < size_smallest_group
                ):
                    if X_test[X_test[column_metric] == group].shape[0] > ns:
                        size_smallest_group = X_test[
                            X_test[column_metric] == group
                        ].shape[0]
                        smallest_group = group

        else:
            if idx == 0:
                size_smallest_group = X_test[X_test[column_metric] == 0].shape[0]
                smallest_group = group
            else:
                if (
                    X_test[X_test[column_metric] == group].shape[0]
                    > size_smallest_group
                ):
                    size_smallest_group = X_test[X_test[column_metric] == group].shape[
                        0
                    ]
                    smallest_group = group

    return smallest_group


def uncertainty(
    column_metric,
    ctgans,
    X_train,
    X_test,
    X_oracle,
    trained_model_dict,
    random_state=0,
    n_samples=5000,
    test_group="small",
):
    from sklearn.metrics import f1_score
    from fairlearn.metrics import equalized_odds_ratio as eo_ratio
    from fairlearn.metrics import demographic_parity_ratio as dp_ratio
    from tqdm import tqdm

    score = "acc"

    groups = list(np.unique(Data[column_metric]))
    orig_random_state = deepcopy(random_state)

    coverage_dict = {}
    width_dict = {}
    excess_dict = {}
    deficet_dict = {}

    for model in ["rf"]:  

        random_state = orig_random_state

  
        try:
            trained_model_dict[model].predict_proba(X_train.drop("y", axis=1))
        except:
            continue

        test_coverage = []
        synth_coverage = []
        synth_coverage5 = []
        synth_coverage10 = []
        blm_coverage = []

        test_width = []
        synth_width = []
        synth_width5 = []
        synth_width10 = []
        blm_width = []

        test_excess = []
        synth_excess = []
        synth_excess5 = []
        synth_excess10 = []
        blm_excess = []

        test_deficet = []
        synth_deficet = []
        synth_deficet5 = []
        synth_deficet10 = []
        blm_deficet = []

        error_test = []
        error_synth = []

        for random_state in tqdm(range(20)):

            _, X_test = train_test_split(
                X_train, test_size=0.2, random_state=random_state
            )

            deepcopy(X_test)

        
            blm_accs = {}

       
            smallest_group = get_group(
                X_test, column_metric, groups, test_group="small"
            )
            print(f"Smallest group: {smallest_group}, random_state: {random_state}")

            for group in groups:

                if group != smallest_group:
                    continue

                # n_samples = 1000
                if group == 0:
                    total_samples = X_test[X_test[column_metric] == 0].shape[0]
                else:
                    total_samples = n_samples

                test_data = X_test[X_test[column_metric] == group]

                oracle_data = X_oracle[X_oracle[column_metric] == group]

                print(f"TEST SIZE {group} - {test_data.shape[0]}")

                if test_data.shape[0] == 0:
                    accuracy = 0

                else:
                    slack = 0.025  # slack for the confidence interval
                    clf = model_dict[model]

                    ############################################################
                    # Oracle
                    ############################################################
                    y_pred = clf.predict(oracle_data.drop("y", axis=1))
                    accuracy_oracle = accuracy_score(oracle_data["y"], y_pred)
                    print(oracle_data.shape, accuracy_oracle)

                    ############################################################
                    # Bootstrap Dtest
                    ############################################################
                    y_pred = clf.predict(test_data.drop("y", axis=1))
                    accuracy = accuracy_score(test_data["y"], y_pred)
                    R = 1000
                    confidence_level = 0.95
                    metric = accuracy_score
                    scores = bootstrap(
                        test_data["y"].values.reshape(-1, 1),
                        y_pred.reshape(-1, 1),
                        metric,
                        R,
                    )
                    bottom_test, top_test = confidence_intervals(
                        scores, confidence_level
                    )
                    score = 0
                    if (
                        accuracy_oracle >= bottom_test.values - slack
                        and accuracy_oracle <= top_test.values + slack
                    ):
                        score = 1
                    test_coverage.append(score)
                    error_test.append(accuracy_oracle - accuracy)
                    excess, deficet, width = compute_interval_metrics(
                        lb=bottom_test.values - slack,
                        ub=top_test.values + slack,
                        true=accuracy_oracle,
                    )
                    test_width.append(width)
                    if excess != -9999:
                        test_excess.append(excess)
                    if deficet != -9999:
                        test_deficet.append(deficet)

                    ############################################################
                    # MBM Interval
                    ############################################################
                    X_test_new = deepcopy(X_test)
                    X_test_new["S"] = trained_model_dict[model].predict_proba(
                        X_test_new.drop("y", axis=1)
                    )[:, 1]

                    model3 = brm(
                        "S ~ Gender + Ethnicity + Education + Country + y", X_test_new
                    )
                    fit3 = model3.fit(
                        backend=pyro_backend, seed=0, iter=1000, warmup=100
                    )
                    myscores = fit3.fitted(what="sample", data=None, seed=0)

                    blm_accs = []
                    for j in range(100):
                        scores3 = np.array(
                            [
                                myscores[np.random.randint(1000, size=1), i]
                                for i in range(myscores.shape[1])
                            ]
                        )
                        y_pred = (scores3 > 0.75).astype(int).flatten()
                        group_ids = np.argwhere(
                            np.array(X_test_new[column_metric] == group).astype(int)
                            == 1
                        )
                        y_pred_group = y_pred[group_ids]
                        acc = accuracy_score(test_data["y"], y_pred_group)
                        blm_accs.append(acc)

                    mean_blm_acc = np.mean(blm_accs)
                    std_blm_acc = np.std(blm_accs)
                    bottom_blm = mean_blm_acc - 1.96 * std_blm_acc
                    top_blm = mean_blm_acc + 1.96 * std_blm_acc

                    score = 0
                    if (
                        accuracy_oracle >= bottom_blm - slack
                        and accuracy_oracle <= top_blm + slack
                    ):
                        score = 1
                    blm_coverage.append(score)
                    excess, deficet, width = compute_interval_metrics(
                        lb=bottom_blm - slack, ub=top_blm + slack, true=accuracy_oracle
                    )
                    blm_width.append(width)
                    if excess != -9999:
                        blm_excess.append(excess)
                    if deficet != -9999:
                        blm_deficet.append(deficet)

                    ############################################################
                    # 3S (k=1)
                    ############################################################
                    syn_accs = []
                    for i in range(100):
                        shift_df, _ = ctgans[0].sample(
                            1,
                            shift=False,
                            condition_column=column_metric,
                            condition_value=group,
                        )
                        count = 0
                        while shift_df.shape[0] <= total_samples:
                            generated_tmp, _ = ctgans[0].sample(
                                n_samples,
                                shift=False,
                                condition_column=column_metric,
                                condition_value=group,
                            )
                            tmp_df = generated_tmp[
                                generated_tmp[column_metric] == group
                            ]
                            shift_df = shift_df.append(tmp_df)
                            count += 1

                        syn_data = shift_df[shift_df[column_metric] == group]
                        y_pred = clf.predict(syn_data.drop("y", axis=1))
                        accuracy = accuracy_score(syn_data["y"], y_pred)
                        syn_accs.append(accuracy)

                    mean_syn_acc = np.mean(syn_accs)
                    std_syn_acc = np.std(syn_accs)
                    bottom_syn = mean_syn_acc - 1.96 * std_syn_acc
                    top_syn = mean_syn_acc + 1.96 * std_syn_acc

                    ############################################################
                    # 3S (k=5)
                    ############################################################
                    syn_accs5 = []
                    for ctgan in ctgans[0:5]:
                        shift_df, _ = ctgan.sample(
                            1,
                            shift=False,
                            condition_column=column_metric,
                            condition_value=group,
                        )

                        count = 0
                        while shift_df.shape[0] <= total_samples:
                            generated_tmp, _ = ctgan.sample(
                                n_samples,
                                shift=False,
                                condition_column=column_metric,
                                condition_value=group,
                            )
                            tmp_df = generated_tmp[
                                generated_tmp[column_metric] == group
                            ]
                            shift_df = shift_df.append(tmp_df)
                            count += 1

                        syn_data = shift_df[shift_df[column_metric] == group]
                        y_pred = clf.predict(syn_data.drop("y", axis=1))
                        accuracy = accuracy_score(syn_data["y"], y_pred)
                        syn_accs5.append(accuracy)

                    mean_syn_acc = np.mean(syn_accs5)
                    std_syn_acc = np.std(syn_accs5)
                    bottom_syn5 = mean_syn_acc - 1.96 * std_syn_acc
                    top_syn5 = mean_syn_acc + 1.96 * std_syn_acc

                    ############################################################
                    # 3S (k=10)
                    ############################################################
                    syn_accs10 = []
                    for ctgan in ctgans:
                        shift_df, _ = ctgan.sample(
                            1,
                            shift=False,
                            condition_column=column_metric,
                            condition_value=group,
                        )

                        count = 0
                        while shift_df.shape[0] <= total_samples:
                            generated_tmp, _ = ctgan.sample(
                                n_samples,
                                shift=False,
                                condition_column=column_metric,
                                condition_value=group,
                            )
                            tmp_df = generated_tmp[
                                generated_tmp[column_metric] == group
                            ]
                            shift_df = shift_df.append(tmp_df)
                            count += 1

                        syn_data = shift_df[shift_df[column_metric] == group]
                        y_pred = clf.predict(syn_data.drop("y", axis=1))
                        accuracy = accuracy_score(syn_data["y"], y_pred)
                        syn_accs10.append(accuracy)

                    mean_syn_acc = np.mean(syn_accs10)
                    std_syn_acc = np.std(syn_accs10)
                    error_synth.append(accuracy_oracle - mean_syn_acc)
                    bottom_syn10 = mean_syn_acc - 1.96 * std_syn_acc
                    top_syn10 = mean_syn_acc + 1.96 * std_syn_acc

                    # compute if the accuracy_oracle is within the confidence interval of the accuracy
                    score = 0
                    if (
                        accuracy_oracle >= bottom_syn - slack
                        and accuracy_oracle <= top_syn + slack
                    ):
                        score = 1

                    synth_coverage.append(score)
                    excess, deficet, width = compute_interval_metrics(
                        lb=bottom_syn - slack, ub=top_syn + slack, true=accuracy_oracle
                    )
                    synth_width.append(width)
                    if excess != -9999:
                        synth_excess.append(excess)
                    if deficet != -9999:
                        synth_deficet.append(deficet)

                    score = 0
                    if (
                        accuracy_oracle >= bottom_syn5 - slack
                        and accuracy_oracle <= top_syn5 + slack
                    ):
                        score = 1
                    synth_coverage5.append(score)
                    excess, deficet, width = compute_interval_metrics(
                        lb=bottom_syn5 - slack,
                        ub=top_syn5 + slack,
                        true=accuracy_oracle,
                    )
                    synth_width5.append(width)
                    if excess != -9999:
                        synth_excess5.append(excess)
                    if deficet != -9999:
                        synth_deficet5.append(deficet)

                    score = 0
                    if (
                        accuracy_oracle >= bottom_syn10 - slack
                        and accuracy_oracle <= top_syn10 + slack
                    ):
                        score = 1
                    synth_coverage10.append(score)
                    excess, deficet, width = compute_interval_metrics(
                        lb=bottom_syn10 - slack,
                        ub=top_syn10 + slack,
                        true=accuracy_oracle,
                    )
                    synth_width10.append(width)
                    if excess != -9999:
                        synth_excess10.append(excess)
                    if deficet != -9999:
                        synth_deficet10.append(deficet)

        mean_test_coverage = np.mean(test_coverage)
        mean_blm_coverage = np.mean(blm_coverage)
        mean_synth_coverage = np.mean(synth_coverage)
        mean_synth_coverage5 = np.mean(synth_coverage5)
        mean_synth_coverage10 = np.mean(synth_coverage10)
        np.mean(error_test)
        np.mean(error_synth)

        # mean for width for synth, synth2, synth3, test, mbm
        mean_synth_width = np.mean(synth_width)
        mean_synth_width5 = np.mean(synth_width5)
        mean_synth_width10 = np.mean(synth_width10)
        mean_test_width = np.mean(test_width)
        mean_blm_width = np.mean(blm_width)

        # excess for width for synth, synth2, synth3, test, mbm
        mean_synth_excess = np.mean(synth_excess)
        mean_synth_excess5 = np.mean(synth_excess5)
        mean_synth_excess10 = np.mean(synth_excess10)
        mean_test_excess = np.mean(test_excess)
        mean_blm_excess = np.mean(blm_excess)

        # mean for excess for synth, synth2, synth3, test, mbm
        mean_synth_deficet = np.mean(synth_deficet)
        mean_synth_deficet5 = np.mean(synth_deficet5)
        mean_synth_deficet10 = np.mean(synth_deficet10)
        mean_test_deficet = np.mean(test_deficet)
        mean_blm_deficet = np.mean(blm_deficet)

        # coverage dict
        coverage_dict[model] = {
            "test": mean_test_coverage,
            "synth": mean_synth_coverage,
            "synth2": mean_synth_coverage5,
            "synth3": mean_synth_coverage10,
            "MBM": mean_blm_coverage,
        }

        # width dict
        width_dict[model] = {
            "test": mean_test_width,
            "synth": mean_synth_width,
            "synth2": mean_synth_width5,
            "synth3": mean_synth_width10,
            "MBM": mean_blm_width,
        }

        # excess dict
        excess_dict[model] = {
            "test": mean_test_excess,
            "synth": mean_synth_excess,
            "synth2": mean_synth_excess5,
            "synth3": mean_synth_excess10,
            "MBM": mean_blm_excess,
        }

        # deficet dict
        deficet_dict[model] = {
            "test": mean_test_deficet,
            "synth": mean_synth_deficet,
            "synth2": mean_synth_deficet5,
            "synth3": mean_synth_deficet10,
            "MBM": mean_blm_deficet,
        }

    return coverage_dict, width_dict, excess_dict, deficet_dict


# Run coverage uncertainty exp

In [ ]:
from tqdm import tqdm

try:
    coverage, width, excess, deficet = uncertainty(
        column_metric=column_metric,
        ctgans=ctgans[0:10],
        X_train=X_train,
        X_test=X_test,
        X_oracle=X_oracle,
        trained_model_dict=trained_model_dict,
        random_state=i * 100,
        test_group="small",
    )
    done = True

except Exception:
    import traceback

    print(traceback.format_exc())


In [ ]:
dname = "drug"
folder = "../results"
for model in list(coverage.keys()):
    mdf_coverage = pd.DataFrame.from_dict(
        coverage[model], orient="index", columns=["Value"]
    )
    mdf_coverage.to_csv(f"{folder}/coverage_{dname}_{model}.csv")

for model in list(width.keys()):
    mdf_width = pd.DataFrame.from_dict(width[model], orient="index", columns=["Value"])
    mdf_width.to_csv(f"{folder}/width_{dname}_{model}.csv")

for model in list(excess.keys()):
    mdf_excess = pd.DataFrame.from_dict(
        excess[model], orient="index", columns=["Value"]
    )
    mdf_excess.to_csv(f"{folder}/excess_{dname}_{model}.csv")

for model in list(deficet.keys()):
    mdf_deficit = pd.DataFrame.from_dict(
        deficet[model], orient="index", columns=["Value"]
    )
    mdf_deficit.to_csv(f"{folder}/deficet_{dname}_{model}.csv")
